In [1]:
%load_ext lab_black

In [2]:
import math
import os
import random
import shutil
from pathlib import Path
from typing import Callable, List, Sequence, Tuple

import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision.models import mobilenet_v2
from torchvision.models.detection import fasterrcnn_resnet50_fpn

In [3]:
box_model = fasterrcnn_resnet50_fpn(pretrained=True, progress=False)

---

## Reproduce on CPU

In [4]:
xs = torch.rand(2, 3, 1080, 1920, dtype=torch.float32)

In [5]:
ys = [
    {
        "labels": torch.tensor([1], dtype=torch.int64),
        "boxes": torch.tensor([[956.0000, 316.3117, 1134.0000, 838.8275]], dtype=torch.float32),
    },
    {
        "labels": torch.tensor([1], dtype=torch.int64),
        "boxes": torch.tensor([[956.0000, 316.3117, 1134.0000, 838.8275]], dtype=torch.float32),
    },
]

In [6]:
ys

[{'labels': tensor([1]),
  'boxes': tensor([[ 956.0000,  316.3117, 1134.0000,  838.8275]])},
 {'labels': tensor([1]),
  'boxes': tensor([[ 956.0000,  316.3117, 1134.0000,  838.8275]])}]

In [7]:
box_model(xs, ys)

{'loss_classifier': tensor(0.0447, grad_fn=<NllLossBackward>),
 'loss_box_reg': tensor(0.0080, grad_fn=<DivBackward0>),
 'loss_objectness': tensor(0.0492, grad_fn=<BinaryCrossEntropyWithLogitsBackward>),
 'loss_rpn_box_reg': tensor(0.0049, grad_fn=<DivBackward0>)}

---

## On GPU

In [8]:
xs = torch.rand(2, 3, 1080, 1920, dtype=torch.float32).cuda()

In [9]:
ys = [
    {
        "labels": torch.tensor([1], dtype=torch.int64).cuda(),
        "boxes": torch.tensor([[956.0000, 316.3117, 1134.0000, 838.8275]], dtype=torch.float32).cuda(),
    },
    {
        "labels": torch.tensor([1], dtype=torch.int64).cuda(),
        "boxes": torch.tensor([[956.0000, 316.3117, 1134.0000, 838.8275]], dtype=torch.float32).cuda(),
    },
]

In [10]:
ys

[{'labels': tensor([1], device='cuda:0'),
  'boxes': tensor([[ 956.0000,  316.3117, 1134.0000,  838.8275]], device='cuda:0')},
 {'labels': tensor([1], device='cuda:0'),
  'boxes': tensor([[ 956.0000,  316.3117, 1134.0000,  838.8275]], device='cuda:0')}]

In [11]:
box_model = box_model.cuda()

In [12]:
box_model(xs, ys)

{'loss_classifier': tensor(0.0441, device='cuda:0', grad_fn=<NllLossBackward>),
 'loss_box_reg': tensor(0.0081, device='cuda:0', grad_fn=<DivBackward0>),
 'loss_objectness': tensor(0.0523, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>),
 'loss_rpn_box_reg': tensor(0.0047, device='cuda:0', grad_fn=<DivBackward0>)}